### Open File

In [1]:
import os
import pandas as pd
import re
pd.set_option('display.max_rows', None)

In [2]:
os.chdir('.\\data\\data')

In [3]:
commit = pd.read_csv("xplore malang.csv")

In [4]:
os.chdir('..\\xplore-issues')

In [5]:
issues = pd.read_csv("xplore-issues.csv")
logs = pd.read_csv("xplore-logs.csv")

### Log content

In [6]:
commit.head()

,Author,Commit,Message,Date
0,Hanif Ramadhan,2681e9b,Final,4/29/2019
1,dennis cunong,3d514ee,Rotate Landscape Feed,4/26/2019
2,Ilham Satria,5cbeff8,Rotate Landscape Home,4/26/2019
3,aprilia widyas,16ce644,intent setting,4/26/2019
4,Hanif Ramadhan,cf73c41,Merapihkan Kodingan,4/26/2019


In [7]:
issues.head()

,status,issue,title,date,assignee
0,closed,61,[3][4] Membuat back end dari fitur Profil,2019-04-09T03:09:05.998785+00:00,aulia ikhsanati
1,closed,63,[3][7] Membuat front end dari kategori Transpo...,2019-04-09T03:16:33.782299+00:00,aulia ikhsanati
2,closed,64,[3][8] Membuat back end dari fitur Transportasi,2019-04-21T09:11:21.552223+00:00,Rizka Fadhilah
3,closed,67,[3][10] Membuat back end dari fitur Filter,2019-04-30T03:29:32.737889+00:00,Hanif Ramadhan
4,closed,66,[3][9] Membuat front end dari fitur Filter,2019-04-30T03:27:48.782392+00:00,Ilham Satria


In [8]:
logs.head()

,issue,status,from,user,date
0,4,resolved,new,Rizka Fadhilah,2019-02-25T03:30:06.239858+00:00
1,6,resolved,new,aprilia widyas,2019-02-25T10:10:47.328461+00:00
2,5,resolved,new,aprilia widyas,2019-02-25T12:26:12.251917+00:00
3,13,resolved,new,aprilia widyas,2019-03-03T17:51:50.514957+00:00
4,8,closed,new,ekky novriza alam,2019-03-19T04:15:47.211468+00:00


### Declare Final Frame

In [9]:
finalFrame = pd.DataFrame(columns = ['Case ID', 'Timestamp', 'Activity', 'Resource'])

### Open Issue

In [10]:
openissue = [0] * len(issues['issue'])
for items in openissue:
    openissue[openissue.index(items)] = "Open"

In [11]:
openissue = pd.DataFrame({'Case ID': issues['issue'],'Timestamp': issues['date'],'Activity': openissue, 'Resource': issues['assignee']})
finalFrame = finalFrame.append(openissue)

In [12]:
finalFrame['Timestamp'] = pd.to_datetime(finalFrame['Timestamp'])

### Log Issue

In [13]:
logsact = []
for items in logs['status']:
    logsact.append(items.capitalize())

In [14]:
logissue = pd.DataFrame({'Case ID': logs['issue'],'Timestamp': logs['date'],'Activity': logsact, 'Resource': logs['user']})
finalFrame = finalFrame.append(logissue)

### Commit dengan issue reference

In [15]:
commit['Date'] = pd.to_datetime(commit['Date'])

#### menambahkan no. issue ke list

In [16]:
commitMessage = commit['Message']
knownCommit = [0] * len(commitMessage)
for items in commitMessage:
    if(re.findall("#[\d]", items)):
        hsh = re.sub("[^#\d]", "", items)
        hsh = re.sub("#", "", hsh)
        knownCommit[list(commitMessage).index(items)] = hsh

### Commit tanpa issue reference

In [17]:
unknownCommit = []
for items in commitMessage:
    if(not re.findall("[(#\d)]", items)):
        unknownCommit.append(items)
    else:
        unknownCommit.append('null')
        

In [18]:
issueList = issues['title']
issueId = issues['issue']
issueList = [re.sub('[\[\]\d]', '', x) for x in issueList]
issueList = [x.strip() for x in issueList]

### Case Folding

In [19]:
unknownCommit = [x.lower() for x in unknownCommit]
issueList = [x.lower() for x in issueList]

### Stopwords Removal

In [20]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
 
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

unknownCommit = [stopword.remove(x) for x in unknownCommit]

In [21]:
issueList = [stopword.remove(x) for x in issueList]

### URL Removal

In [22]:
unknownCommit = [re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in unknownCommit]
unknownCommit = ['merge' if re.findall('^m ', x) else x for x in unknownCommit]

### Cleaning

In [23]:
unknownCommit = [x.replace("front-end","front end") for x in unknownCommit]
unknownCommit = [x.replace("frontend","front end") for x in unknownCommit]
unknownCommit = [x.replace("back-end","back end") for x in unknownCommit]
unknownCommit = [x.replace("backend","back end") for x in unknownCommit]

In [24]:
issueList = [x.replace("front-end","front end") for x in issueList]
issueList = [x.replace("frontend","front end") for x in issueList]
issueList = [x.replace("back-end","back end") for x in issueList]
issueList = [x.replace("backend","back end") for x in issueList]

### Compare commit dengan issue

In [25]:
unknownId = [0] * len(unknownCommit)
for items in unknownCommit:
    for issue in issueList:
        if items in issue:
            unknownId[unknownCommit.index(items)] = issueId[issueList.index(issue)]

#### menambahkan no. issue ke list

In [26]:
for items in knownCommit:
    if items == 0:
        knownCommit[knownCommit.index(items)] = unknownId[knownCommit.index(items)]

### Merge dataset

In [27]:
commitMsg = [0] * len(commit['Message'])
for items in list(commit['Message']):
    if "merge" in items.lower():
        commitMsg[list(commit['Message']).index(items)] = "Merge Branch"
    elif "test" in items.lower():
        commitMsg[list(commit['Message']).index(items)] = "Testing"
    else:
        commitMsg[list(commit['Message']).index(items)] = "Implement task"
        

In [28]:
commitissue = pd.DataFrame({'Case ID': knownCommit,'Timestamp': commit['Date'],'Activity': commitMsg, 'Resource': commit['Author']})
finalFrame = finalFrame.append(commitissue)

In [29]:
finalFrame['Timestamp'] = pd.to_datetime(finalFrame['Timestamp'], utc=True)

In [30]:
finalFrame.sort_values(by=['Timestamp'], inplace=True, ascending=True)

In [31]:
os.chdir('..\\hasil')
finalFrame.to_csv('xplore.csv', index=False)